<a href="https://colab.research.google.com/github/tpunamiya/tpunamiya.github.io/blob/main/w03_8Puzzle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Example: 8 Puzzle Problems

![](https://ece.uwaterloo.ca/~dwharder/aads/Algorithms/N_puzzles/images/puz3.png)

A sliding tile puzzle where you can swap the blank with an adjacent piece, trying to reach a **goal configuration**. The cells are numbered 0 to 8, starting at the top left and going row by row left to right. The pieces are numbered 1 to 8, with 0 representing the blank. An **action** is the cell index number that is to be swapped with the blank (*not* the actual number to be swapped but the index into the state). So the diagram above left is the state `(5, 2, 7, 8, 4, 0, 1, 3, 6)`, and the action is `8`, because the cell number 8 (the 9th or last cell, the `6` in the bottom right) is swapped with the blank.

There are two disjoint sets of states that cannot be reached from each other. One set has an even number of **inversions**; the other has an odd number. An **inversion** is when a piece in the state is larger than a piece that follows it.

This example shows how to i) model the 8 puzzle problem based on abstract classes for `Problem` and `Node`; ii) apply different heuristic functions; and iii) solve the problem using the **A\* search** algorithm.


In [23]:
# importing necessary libraries
%matplotlib inline
import matplotlib.pyplot as plt
import random
import heapq
import math
import sys
from collections import defaultdict, deque, Counter
from itertools import combinations

## Abstract classes for Problem and Node

In [24]:
# Abstract class Problem
class Problem(object):
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When yiou create an instance of a subclass, specify `initial`, and `goal` states
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds):
        self.__dict__.update(initial=initial, goal=goal, **kwds)
# init is a function that defines a class
    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0

    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)

In [25]:
# Abstract class Node
class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost

# Indicates an algorithm couldn't find a solution
failure = Node('failure', path_cost=math.inf)
# Indicates iterative deepening search was cut off
cutoff  = Node('cutoff',  path_cost=math.inf)

def expand(problem, node):
    "Expand a node, generating the children nodes."
    # retrieve the state of current node
    s = node.state
    # check all available actions at a particular state (node)
    for action in problem.actions(s):
        # next state (s1) after applying the chosen action at the state (s)
        s1 = problem.result(s, action)
        # add the cost to reach state s1
        cost = node.path_cost + problem.action_cost(s, action, s1)
        # return a tuple with the next state, the current node, action and cost
        yield Node(s1, node, action, cost)

# sequence of actions to reach a particular node
# useful for building the solution path
def path_actions(node):
    "The sequence of actions to get to this node."
    if node.parent is None:
        return []
    return path_actions(node.parent) + [node.action]

# sequence of states to reach a particular node
# useful for building the solution path
def path_states(node):
    "The sequence of states to get to this node."
    if node in (cutoff, failure, None):
        return []
    return path_states(node.parent) + [node.state]

## Priority Queue, Best-first search and A* search

In [15]:
FIFOQueue = deque
LIFOQueue = list

# base class implementing a priority queue used for finding the action with minimum cost
class PriorityQueue:
    """A queue in which the item with minimum f(item) is always popped first."""

    def __init__(self, items=(), key=lambda x: x):
        self.key = key
        # a heap of (score, item) pairs
        self.items = []
        for item in items:
            self.add(item)

    def add(self, item):
        """Add item to the queue."""
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)

    def pop(self):
        """Pop and return the item with min f(item) value."""
        return heapq.heappop(self.items)[1]

    def top(self): return self.items[0][1]

    def __len__(self): return len(self.items)

Complete the following best-first search code yourself with hints provided. The complete solution can be found at the end of the notebook.

In [28]:
# (Generic) Best-first search: selects a node for expansion according to an evaluation function
def best_first_search(problem, f):
    "Search nodes with minimum f(node) value first."
    node = Node(problem.initial)
    frontier = PriorityQueue([node], key=f)
    reached = {problem.initial: node}   # reached dictionary format {state: node}
    pruned = set()  # a set of pruned nodes
    # iterate over the nodes in the frontier
    while frontier:
        # pick a node from frontier - based on the PriorityQueue
        node = frontier.pop()

        # if the node is in the pruned set, we ignore this, and move on in the frontier
        if node in pruned:
          continue

        # test for goal state
        if problem.is_goal(node.state):
           return node

        # if not goal, expand all child nodes
        for child in expand(problem, node):
            pass # delete this

            # check expand(problem, node) in class Node
            # retrieve child node's state
            s = child.state

            # if child node state not visited before
            # add child node to visited set
            # add child node to the frontier (for future expansion)
            if s not in reached:
              reached[s] = child
              frontier.add(child)


            # elseif child node state has been visited before but our new path cost is smaller
            # add the original node (visited before) to the pruned set
            # update state: child node in dictionary
            # add child node to the frontier (for future expansion)
            elif child.path_cost < reached[s].path_cost:
              pruned.add(reached[s])
              reached[s] = child
              frontier.add(child)

    return failure

In [17]:
# helper function used by A* for keeping track of the actual path cost
def g(n): return n.path_cost

Complete the following A* search code. The complete solution can be found at the end of the notebook.



In [29]:
# A* method with parameterizable heuristic h, which is either supplied externally, or defined within the problem class
def astar_search(problem, h=None):
    """Search nodes with minimum f(n) = g(n) + h(n)."""
    h = h or problem.h
    return best_first_search(problem, f=lambda n: g(n) + h(n))

## Problem-specific class - EightPuzzle


Complete the Manhattan distance heuristic in `def h2(self, node)` below. The solution can be found at the end of the notebook.

In [31]:
# Problem-specific class - EightPuzzle
class EightPuzzle(Problem):
    """ The problem of sliding tiles numbered from 1 to 8 on a 3x3 board,
    where one of the squares is a blank, trying to reach a goal configuration.
    A board state is represented as a tuple of length 9, where the element at index i
    represents the tile number at index i, or 0 if for the empty square, e.g. the goal:
        1 2 3
        4 5 6 ==> (1, 2, 3, 4, 5, 6, 7, 8, 0)
        7 8 _
    """

    # we can modify the desired goal state to any other puzzle configuration
    def __init__(self, initial, goal=(0, 1, 2, 3, 4, 5, 6, 7, 8)):
        # parity check
        assert inversions(initial) % 2 == inversions(goal) % 2
        self.initial, self.goal = initial, goal

    def actions(self, state):
        """The indexes of the squares that the blank can move to. 0 is top left corner, etc."""
        moves = ((1, 3),    (0, 2, 4),    (1, 5),
                 (0, 4, 6), (1, 3, 5, 7), (2, 4, 8),
                 (3, 7),    (4, 6, 8),    (7, 5))
        blank = state.index(0)
        return moves[blank]

    def result(self, state, action):
        """Swap the blank with the square numbered `action`."""
        s = list(state)
        blank = state.index(0)
        s[action], s[blank] = s[blank], s[action]
        return tuple(s)

    def h1(self, node):
        """The misplaced tiles heuristic. All out: 8."""
        return sum(s != g for (s, g) in zip(node.state, self.goal) if s != 0)

    def h2(self, node):
        """The Manhattan distance heuristic."""
        pass   # delete this
        # <YOUR CODE HERE>


    # Choose h2 to be the default heuristic. You can try h1 as well.
    def h(self, node):
      return self.h2(node)



# helper functions
def inversions(board):
    "The number of times a piece is a smaller number than a following piece."
    return sum((a > b and a != 0 and b != 0) for (a, b) in combinations(board, 2))

# A string representing of an 8-puzzle board
def board8(board, fmt=(3 * '{} {} {}\n')):
    "A string representing of an 8-puzzle board"
    return fmt.format(*board).replace('0', '_')

# board implementation - for visualisation
class Board(defaultdict):
    empty = '.'
    off = '#'

    def __init__(self, board=None, width=8, height=8, to_move=None, **kwds):
        if board is not None:
            self.update(board)
            self.width, self.height = (board.width, board.height)
        else:
            self.width, self.height = (width, height)
        self.to_move = to_move

    def __missing__(self, key):
        x, y = key
        if x < 0 or x >= self.width or y < 0 or y >= self.height:
            return self.off
        else:
            return self.empty

    def __repr__(self):
        def row(y): return ' '.join(self[x, y] for x in range(self.width))
        return '\n'.join(row(y) for y in range(self.height))

    def __hash__(self):
        return hash(tuple(sorted(self.items()))) + hash(self.to_move)

In [32]:
# Some specific EightPuzzle problems
e1 = EightPuzzle((1, 4, 2, 0, 7, 5, 3, 6, 8))
e2 = EightPuzzle((1, 2, 3, 4, 5, 6, 7, 8, 0))
e3 = EightPuzzle((4, 0, 2, 5, 1, 3, 7, 8, 6))
e4 = EightPuzzle((7, 2, 4, 5, 0, 6, 8, 3, 1))
e5 = EightPuzzle((8, 6, 7, 2, 5, 4, 3, 0, 1))

In [33]:
# Solve an 8 puzzle problems and count the number of moves
len(path_states(astar_search(e5))) - 1   # why do we need to -1 here?

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

In [30]:
# Print out each state
for s in path_states(astar_search(e1)):
    print(board8(s))

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

## Solutions

Best-first search code

```
# (Generic) Best-first search: selects a node for expansion according to an evaluation function
def best_first_search(problem, f):
    "Search nodes with minimum f(node) value first."
    node = Node(problem.initial)
    frontier = PriorityQueue([node], key=f)
    reached = {problem.initial: node}   # reached dictionary format {state: node}
    pruned = set()  # a set of pruned nodes
    # iterate over the nodes in the frontier
    while frontier:
        # pick a node from frontier - based on the PriorityQueue
        node = frontier.pop()

        # if the node is in the pruned set, we ignore this, and move on in the frontier
        if node in pruned:
            continue
      
        # test for goal state
        if problem.is_goal(node.state):
            return node

        # if not goal, expand all child nodes
        for child in expand(problem, node):
            
            # retrieve child node's state
            s = child.state
            
            # if child node state not visited before
            if s not in reached:
                
                # add child node to visited set
                reached[s] = child
                # add child node to the frontier (for future expansion)
                frontier.add(child)
            
            # elseif child node state has been visited before but our new path cost is smaller
            elif child.path_cost < reached[s].path_cost:

                    # add the original node (visited before) to the pruned set
                    pruned.add(reached[s])

                    # update state: child node in dictionary
                    reached[s] = child
                    # add child node to the frontier (for future expansion)
                    frontier.add(child)

    return failure
  ```

A* search code

```
# A* method with parameterizable heuristic h, which is either supplied externally, or defined within the problem class
def astar_search(problem, h=None):
    """Search nodes with minimum f(n) = g(n) + h(n)."""
    h = h or problem.h
    return best_first_search(problem, f=lambda n: g(n) + h(n))
```

The Manhattan distance heuristic

```
def h2(self, node):
        """The Manhattan distance heuristic."""
        X = (0, 1, 2, 0, 1, 2, 0, 1, 2)
        Y = (0, 0, 0, 1, 1, 1, 2, 2, 2)
        return sum(abs(X[s] - X[g]) + abs(Y[s] - Y[g])
                   for (s, g) in zip(node.state, self.goal) if s != 0)
```